In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
df.annual_income = pd.to_numeric(df.annual_income, errors='coerce')

In [7]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [8]:
categorical = ['lead_source', 'industry', 'employment_status', 'location'] 
numerical = ['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [9]:
df[categorical] = df[categorical] .fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

In [10]:
df[categorical].industry.mode()[0]

'retail'

In [11]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [14]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [15]:
df_train_categorical = df_train.loc[:, df_train.dtypes[df_train.dtypes == 'object'].index]
df_train_numerical = df_train.loc[:, df_train.dtypes[df_train.dtypes != 'object'].index]

In [16]:
from sklearn.metrics import mutual_info_score

In [17]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

In [18]:
mi = df_train[categorical].apply(mutual_info_converted_score)
round(mi.sort_values(ascending=False), 3)

lead_source          0.035
employment_status    0.013
industry             0.012
location             0.004
dtype: float64

In [19]:
from sklearn.feature_extraction import DictVectorizer

In [20]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [22]:
model.intercept_[0]

np.float64(-0.06914728027824993)

In [23]:
model.coef_[0].round(3)

array([-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
       -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
        0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
       -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454])

In [24]:
y_pred = model.predict_proba(X_val)[:, 1]

In [25]:
conv_decision = (y_pred >= 0.5)

In [26]:
round((y_val == conv_decision).mean(), 3)

np.float64(0.7)

In [31]:
def evaluate_without_feature(feature_to_exclude):
    
    feature_names = dv.get_feature_names_out()
    
    # Find indices of features to keep (all except the ones to exclude)
    exclude_indices = [i for i, feat in enumerate(feature_names) if feature_to_exclude in feat]
    keep_indices = [i for i in range(len(feature_names)) if i not in exclude_indices]
    
    # Create new datasets without the feature
    X_train_reduced = X_train[:, keep_indices]
    X_val_reduced = X_val[:, keep_indices]
    
    # Train the model
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_reduced, y_train)
    
    # Evaluate on validation set
    y_pred = model.predict(X_val_reduced)
    accuracy = (y_val == (y_pred >= 0.5)).mean()
    
    return accuracy

In [33]:
features_to_test = ['industry', 'employment_status', 'lead_score']
results = {}

accuracy_baseline = (y_val == conv_decision).mean()
for feature in features_to_test:
    accuracy_without_feature = evaluate_without_feature(feature)
    difference = accuracy_baseline - accuracy_without_feature
    results[feature] = {
        'accuracy': accuracy_without_feature,
        'difference': difference
    }
    print(f"Without {feature}: Accuracy = {accuracy_without_feature:.6f}, Difference = {difference:.6f}")

# Find the feature with the smallest difference
min_feature = min(results, key=lambda x: abs(results[x]['difference']))
print(f"\nFeature with smallest impact: {min_feature}, Difference: {results[min_feature]['difference']:.6f}")

Without industry: Accuracy = 0.699659, Difference = 0.000000
Without employment_status: Accuracy = 0.696246, Difference = 0.003413
Without lead_score: Accuracy = 0.706485, Difference = -0.006826

Feature with smallest impact: industry, Difference: 0.000000


In [38]:
C_param = [0.01, 0.1, 1, 10, 100]
accuracy = []
for c in C_param:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy.append(round((y_val == (y_pred >= 0.5)).mean(), 2))
print(accuracy)

[np.float64(0.7), np.float64(0.7), np.float64(0.7), np.float64(0.7), np.float64(0.7)]
